In [11]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np


# getting info from sports reference

In [3]:
def get_winners (link):
    print (link)
    page = requests.get(link)
    #print('here')
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=0
    k=0
    winners=[]
    for thing in list(range(1,100)):
        if k == 22:
            break
        elif k == 0 or k== 1 or k==2 or k==3 or k==5 or k==6 or k==7 or k==8 or k==10 or k==11 or k==12 or k==13 or k==15 or k==16 or k==17 or k==18 or k==20 or k==21:
            try:
                winners.append(idk[k].select('div')[i].select('div[class="winner"]')[0].select('span')[0].string)
                i+=3
            except:
                i=0
                k+=1
                continue
        else:
            i=0
            k+=1
            continue
    winners_int=[]
    for thing in winners:
        winners_int.append(int(thing))
    return winners_int

In [4]:
#function for getting the teams
#gets the teams that played each other all together
def get_teams (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    i=1
    maybe = []
    for thing in list(range(1,100)):
        if i%3 !=0:
            try:
                maybe.append(idk[k].select('div')[i].select('span')[0].string)
                i+=1
            except:
                i+=1
                continue
        else:
            i+=1
            continue
    return maybe



In [5]:
#uses the function to split up the teams that played each other into 2 separate lists
def split_up_teams (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    first_team = All_teams[0::2]
    second_team = All_teams[1::2]
    first_team_int = []
    for thing in first_team:
        first_team_int.append(int(thing))
    second_team_int = []
    for thing in second_team:
        second_team_int.append(int(thing))
    first_team_pd = pd.DataFrame(first_team_int)
    second_team_pd = pd.DataFrame(second_team_int)
    point_difference = first_team_pd-second_team_pd
    point_difference_list=point_difference.values.tolist()
    point_diff_abs = []
    for thing in point_difference_list:
        point_diff_abs.append(abs(thing[0]))
    return first_team_int,second_team_int,point_diff_abs



In [6]:
def get_team_names (link,k):
    page = requests.get(link)
    soup = BS(page.content, 'html.parser')
    idk=soup.select('div[class="round"]')
    maybe = []
    for thing in list(range(0,100)):
        try:
            maybe.append(idk[k].select('a[href^=/cbb/schools]')[thing].string)
        except:
            continue
    return maybe



In [7]:
def get_teams_get (link):
    team_names = []
    for thing in list(range(0,23)):
        new=get_team_names (link,thing)
        team_names.extend(new)
    team_names.pop(-1)
    team_names.pop(30)
    team_names.pop(60)
    team_names.pop(90)
    team_names.pop(120)
    return team_names



In [8]:
def split_up_teams_2 (link):
    All_teams = []
    for thing in list(range(0,23)):
        new=get_teams (link,thing)
        All_teams.extend(new)
    All_teams.pop(-1)
    All_teams.pop(30)
    All_teams.pop(60)
    All_teams.pop(90)
    All_teams.pop(120)
    return All_teams

In [9]:
#add in the year
#make the 9999

In [10]:
def everything (link):
    first_team_int = []
    second_team_int = []
    point_diff_abs = []
    winners_int = []

    winners=get_winners (link)
    three_lists=split_up_teams (link)
    first_team_int.extend(three_lists[0])
    second_team_int.extend(three_lists[1])
    point_diff_abs.extend(three_lists[2])
    winners_int.extend(winners)
    #time.sleep(3)
    #print(len(first_team_int), len(second_team_int),len(point_diff_abs), len(winners_int))
    df = pd.DataFrame({"first_team": first_team_int, "second_team": second_team_int,
                       "winner" : winners_int, "point_difference": point_diff_abs})
    result = []
    i = 0
    for thing in list(df['winner']):
        #print(f'this is {i}')
        if list(df['first_team'])[i] > list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is if1')
            result.append(2)
        elif list(df['second_team'])[i] > list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif1')
            result.append(2)
        elif list(df['first_team'])[i] < list(df['second_team'])[i] and list(df['first_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif2')
            result.append(1)
        elif list(df['second_team'])[i] < list(df['first_team'])[i] and list(df['second_team'])[i] == list(df['winner'])[i]:
            #print(f'this is elif3')
            result.append(1)
        else:
            result.append(0)
        i+=1
    df['result']=result
    newdf2 = pd.DataFrame(np.repeat(df.values,2,axis=0))
    newdf2.columns = df.columns
    team_names=get_teams_get (link)
    All_teams=split_up_teams_2(link)
    rankings= []
    for thing in All_teams:
        #print(thing)
        rankings.append(int(thing))
    newdf2['team_name'] = team_names
    newdf2['team_rank'] = rankings
    win = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            win.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['winner'])[i]:
            win.append(1)
        elif list(newdf2['team_rank'])[i] != list(newdf2['winner'])[i]:
            win.append(0)
        i+=1
    newdf2['win']=win
    #if the team has a lower seed number, it is given '1', higher seed number '0'
    higher_or_lower_seed = []
    i = 0
    for thing in list(newdf2['team_rank']):
        if list(newdf2['first_team'])[i] == list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(9999)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] < list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['first_team'])[i] and list(newdf2['first_team'])[i] > list(newdf2['second_team'])[i]:
            higher_or_lower_seed.append(0)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] < list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(1)
        elif list(newdf2['team_rank'])[i] == list(newdf2['second_team'])[i] and list(newdf2['second_team'])[i] > list(newdf2['first_team'])[i]:
            higher_or_lower_seed.append(0)
        i+=1
    newdf2['higher_or_lower_seed']=higher_or_lower_seed
    #newdf2.to_csv(f'{link}')
    newdf2.rename(columns = {'team_name':'name'}, inplace = True)
    newdf2['index']=list(range(0,126))
    return newdf2

In [99]:
newdf2=everything ('https://www.sports-reference.com/cbb/postseason/2017-ncaa.html')

https://www.sports-reference.com/cbb/postseason/2017-ncaa.html


In [100]:
year = []
for thing in newdf2['first_team']:
    year.append(2017)

In [101]:
newdf2['year']=year

In [103]:
newdf2

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year
0,1,16,1,15,1,Villanova,1,1,1,0,2017
1,1,16,1,15,1,Mount St. Mary's,16,0,0,1,2017
2,8,9,8,1,1,Wisconsin,8,1,1,2,2017
3,8,9,8,1,1,Virginia Tech,9,0,0,3,2017
4,5,12,5,7,1,Virginia,5,1,1,4,2017
5,5,12,5,7,1,UNC Wilmington,12,0,0,5,2017
6,4,13,4,9,1,Florida,4,1,1,6,2017
7,4,13,4,9,1,ETSU,13,0,0,7,2017
8,6,11,11,5,2,SMU,6,0,1,8,2017
9,6,11,11,5,2,USC,11,1,0,9,2017


In [109]:
#put this into 
other_team_seed = []
i = 0
for thing in list(newdf2['team_rank']):
    if thing == list(newdf2['second_team'])[i]:
        other_team_seed.append(list(newdf2['first_team'])[i])
    elif thing == list(newdf2['first_team'])[i]:
        other_team_seed.append(list(newdf2['second_team'])[i])
    i+=1
print('made it here')
newdf2['other_team_seed']=other_team_seed

made it here


In [105]:
len(list(newdf2['team_rank']))

126

In [108]:
len(other_team_seed)

0

In [94]:
newdf2['']

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year,other_team
0,1,16,1,15,1,Villanova,1,1,1,0,2018,16
1,1,16,1,15,1,Radford,16,0,0,1,2018,1
2,8,9,9,1,2,Virginia Tech,8,0,1,2,2018,9
3,8,9,9,1,2,Alabama,9,1,0,3,2018,8
4,5,12,5,7,1,West Virginia,5,1,1,4,2018,12
5,5,12,5,7,1,Murray State,12,0,0,5,2018,5
6,4,13,13,9,2,Wichita State,4,0,1,6,2018,13
7,4,13,13,9,2,Marshall,13,1,0,7,2018,4
8,6,11,6,5,1,Florida,6,1,1,8,2018,11
9,6,11,6,5,1,St. Bonaventure,11,0,0,9,2018,6


In [ ]:
newdf2.to_csv('sports_reference_2019')

# getting score info from ESPN

In [13]:
def get_score_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
        
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/scoring-per-game/sort/avgPoints/year/'+thing+'/count/321')
    
    names = []
    PTS = []
    FG_perc = []
    _3p_perc = []
    FT_perc = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    PTS.append(float(idk2[i].select('td')[3].string))
                    FG_perc.append(float(idk2[i].select('td')[5].string))
                    _3p_perc.append(float(idk2[i].select('td')[7].string))
                    FT_perc.append(float(idk2[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    PTS.append(float(thing.select('td')[3].string))
                    FG_perc.append(float(thing.select('td')[5].string))
                    _3p_perc.append(float(thing.select('td')[7].string))
                    FT_perc.append(float(thing.select('td')[9].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    PTS.append(float(idk[i].select('td')[3].string))
                    FG_perc.append(float(idk[i].select('td')[5].string))
                    _3p_perc.append(float(idk[i].select('td')[7].string))
                    FT_perc.append(float(idk[i].select('td')[9].string))
                    i+=1
                except:
                    continue
                    i+=1
    D = {'Name':names, 'PTS':PTS, 'FG_perc':FG_perc, '_3p_perc':_3p_perc, 'FT_perc':FT_perc}
    df2 = pd.DataFrame(data=D)
    
    the_year = []
    for thing in df2['Name']:
        the_year.append(year)
        
    df2['year']=the_year
            


            
    return df2

In [ ]:
df3 = get_score_info (2019)

In [ ]:
df2.to_csv('ESPN_2019')

# ESPN Rebound Data

In [14]:
def get_rebound_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
        
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/rebounds/sort/avgPoints/year/'+thing+'/count/321')
    
    names = []
    ORPG = []
    DRPG = []
    RPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    ORPG.append(float(thing.select('td')[4].string))
                    DRPG.append(float(thing.select('td')[6].string))
                    RPG.append(float(thing.select('td')[8].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    ORPG.append(float(idk2[i].select('td')[4].string))
                    DRPG.append(float(idk2[i].select('td')[6].string))
                    RPG.append(float(idk2[i].select('td')[8].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    ORPG.append(float(thing.select('td')[4].string))
                    DRPG.append(float(thing.select('td')[6].string))
                    RPG.append(float(thing.select('td')[8].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    ORPG.append(float(idk[i].select('td')[4].string))
                    DRPG.append(float(idk[i].select('td')[6].string))
                    RPG.append(float(idk[i].select('td')[8].string))
                    i+=1
                except:
                    continue
                    i+=1
                    
    D = {'Name':names, 'ORPG':ORPG, 'DRPG':DRPG, 'RPG':RPG}
    df2 = pd.DataFrame(data=D)

            
            


            
    return df2

In [ ]:
df4 = get_rebound_info (2019)

# ESPN Field Goals

In [15]:
def get_field_goal_info (year):
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/field-goals/sort/avgPoints/year/'+thing+'/count/321')
        
    names = []
    _2pm = []
    _2pa = []
    _2p_perc = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    _2pm.append(float(thing.select('td')[9].string))
                    _2pa.append(float(thing.select('td')[10].string))
                    _2p_perc.append(float(thing.select('td')[11].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    _2pm.append(float(idk2[i].select('td')[9].string))
                    _2pa.append(float(idk2[i].select('td')[10].string))
                    _2p_perc.append(float(idk2[i].select('td')[11].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    _2pm.append(float(thing.select('td')[9].string))
                    _2pa.append(float(thing.select('td')[10].string))
                    _2p_perc.append(float(thing.select('td')[11].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    _2pm.append(float(idk[i].select('td')[9].string))
                    _2pa.append(float(idk[i].select('td')[10].string))
                    _2p_perc.append(float(idk[i].select('td')[11].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, '_2pm':_2pm, '_2pa':_2pa, '_2p_perc':_2p_perc}
    df2 = pd.DataFrame(data=D)

   


            
    return df2

In [ ]:
df5 = get_field_goal_info (2019)

# ESPN 3-point data

In [16]:
def get_3_point_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/3-points/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    _3pm = []
    _3pa = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    _3pm.append(float(thing.select('td')[4].string))
                    _3pa.append(float(thing.select('td')[5].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    _3pm.append(float(idk2[i].select('td')[4].string))
                    _3pa.append(float(idk2[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    _3pm.append(float(thing.select('td')[4].string))
                    _3pa.append(float(thing.select('td')[5].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    _3pm.append(float(idk[i].select('td')[4].string))
                    _3pa.append(float(idk[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, '_3pm':_3pm, '_3pa':_3pa}
    df2 = pd.DataFrame(data=D)

            
    return df2

In [ ]:
df6=get_3_point_info (2019)

# ESPN assists

In [17]:
def get_assists_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/assists/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    APG = []
    AST_TO = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    APG.append(float(thing.select('td')[4].string))
                    AST_TO.append(float(thing.select('td')[7].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    APG.append(float(idk2[i].select('td')[4].string))
                    AST_TO.append(float(idk2[i].select('td')[7].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    APG.append(float(thing.select('td')[4].string))
                    AST_TO.append(float(thing.select('td')[7].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    APG.append(float(idk[i].select('td')[4].string))
                    AST_TO.append(float(idk[i].select('td')[7].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'APG':APG, 'AST_TO':AST_TO}
    df2 = pd.DataFrame(data=D)

            
    return df2

In [ ]:
df7=get_assists_info (2019)

# ESPN Steals info

In [18]:
def get_steals_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/steals/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    STPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    STPG.append(float(thing.select('td')[4].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    STPG.append(float(idk2[i].select('td')[4].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    STPG.append(float(thing.select('td')[4].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    STPG.append(float(idk[i].select('td')[4].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'STPG':STPG}
    df2 = pd.DataFrame(data=D)

            
    return df2

In [ ]:
df8=get_steals_info (2019)

# ESPN get blocks

In [19]:
def get_blocks_info (year):
    
    list_years = []
    i = year
    for num in range(0,1):
        list_years.append(str(i))
        i+=1
        
    
    links = []
    for thing in list_years:
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing)
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/41')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/81')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/121')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/161')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/201')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/241')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/281')
        links.append('http://www.espn.com/mens-college-basketball/statistics/team/_/stat/blocks/sort/avgPoints/year/'+thing+'/count/321')
    
    
    names = []
    BLKPG = []
    for link in links:
        #print(link)
        page = requests.get(link)
        soup = BS(page.content, 'html.parser')
        idk=soup.select('tr[class^="oddrow"]')
        idk2=soup.select('tr[class^="evenrow"]')
        i = 0
        if len(idk)>len(idk2):
            for thing in idk:
                try:    
                    names.append(thing.select('td')[1].string)
                    BLKPG.append(float(thing.select('td')[5].string))
                    
                    names.append(idk2[i].select('td')[1].string)
                    BLKPG.append(float(idk2[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
        else:
            for thing in idk2:
                try:    
                    names.append(thing.select('td')[1].string)
                    BLKPG.append(float(thing.select('td')[5].string))
                    
                    names.append(idk[i].select('td')[1].string)
                    BLKPG.append(float(idk[i].select('td')[5].string))
                    i+=1
                except:
                    continue
                    i+=1
            
    D = {'Name':names, 'BLKPG':BLKPG}
    df2 = pd.DataFrame(data=D)

            
    return df2

In [ ]:
df9=get_blocks_info (2019)

# Everything for ESPN - Merges everything also

In [20]:
def merge_espn (year):
    df3 = get_score_info (year)
    df4 = get_rebound_info (year)
    df5 = get_field_goal_info (year)
    df6=get_3_point_info (year)
    df7=get_assists_info (year)
    df8=get_steals_info (year)
    df9=get_blocks_info (year)
    df2=pd.merge(df3,df4, how = 'inner', on = 'Name')
    df2=pd.merge(df5,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df6,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df7,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df8,df2, how = 'inner', on = 'Name')
    df2=pd.merge(df9,df2, how = 'inner', on = 'Name')
    return df2

In [ ]:
df2=merge_espn (2019)

In [ ]:
df2

# Advanced Stats Sports Reference

In [190]:
def get_advanced_stats (year):
    page = requests.get('https://www.sports-reference.com/cbb/seasons/'+f'{year}'+'-advanced-school-stats.html')
    soup = BS(page.content, 'html.parser')
    idk = soup.select('tbody')
    
    name_adv=[]
    SRS_adv = []
    SOS_adv = []
    pace_adv = []
    ORtg_adv = []
    eFG_perc_adv = []
    FTr_adv = []
    
    for thing in idk[0].select('tr'):
        try:

            #school name
            name=thing.select('td[data-stat="school_name"]')[0].select('a')[0].string
            name_adv.append(name)

            #SRS
            SRS_adv.append(float(thing.select('td[data-stat="srs"]')[0].string))

            #SOS
            SOS_adv.append(float(thing.select('td[data-stat="sos"]')[0].string))

            #pace
            pace_adv.append(float(thing.select('td[data-stat="pace"]')[0].string))

            #ORtg
            ORtg_adv.append(float(thing.select('td[data-stat="off_rtg"]')[0].string))

            #eFG%
            eFG_perc_adv.append(float(thing.select('td[data-stat="efg_pct"]')[0].string))

            #FTr
            FTr_adv.append(float(thing.select('td[data-stat="fta_per_fga_pct"]')[0].string))
            
        except:
            continue
        
    D = {'name':name_adv,'SRS_adv':SRS_adv,'SOS_adv':SOS_adv,'pace_adv':pace_adv,'ORtg_adv':ORtg_adv,'eFG_perc_adv':eFG_perc_adv}
    adv_stats = pd.DataFrame(data=D)
    
    return adv_stats
        
        

In [192]:
adv_stat=get_advanced_stats (2019)
newdf2=everything ('https://www.sports-reference.com/cbb/postseason/2019-ncaa.html')
perhaps=pd.merge(newdf2, adv_stat, how = 'inner', on='name')
perhaps = perhaps.sort_values(by ='index' )

https://www.sports-reference.com/cbb/postseason/2019-ncaa.html


In [194]:
perhaps

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,SRS_adv,SOS_adv,pace_adv,ORtg_adv,eFG_perc_adv
0,1,16,1,15,1,Duke,1,1,1,0,26.90,11.98,73.2,112.6,0.536
4,1,16,1,15,1,North Dakota State,16,0,0,1,-4.01,-2.07,66.8,108.5,0.540
5,5,12,12,7,2,Mississippi State,5,0,1,4,15.96,9.07,68.6,111.9,0.546
6,5,12,12,7,2,Liberty,12,1,0,5,5.27,-3.88,65.4,112.8,0.569
8,4,13,4,9,1,Virginia Tech,4,1,1,6,19.28,7.79,64.4,113.3,0.556
11,4,13,4,9,1,Saint Louis,13,0,0,7,5.06,2.23,67.0,99.4,0.467
12,6,11,6,5,1,Maryland,6,1,1,8,16.01,10.09,66.6,107.3,0.514
14,6,11,6,5,1,Belmont,11,0,0,9,9.12,-2.60,74.2,116.2,0.580
15,3,14,3,11,1,Yale,14,0,0,11,5.52,-1.24,72.9,109.7,0.556
16,7,10,10,3,2,Louisville,7,0,1,12,17.28,11.04,68.4,107.4,0.509


In [196]:
find_missing_values (list(perhaps['index']))

2
3
10
17
21
26
60
76
84
92
99


[2, 3, 10, 17, 21, 26, 60, 76, 84, 92, 99]

In [197]:
newdf2[(newdf2['index']==2)]

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index
2,8,9,9,1,2,VCU,8,0,1,2


In [ ]:
list(adv_stat['name'])

# Everything

In [258]:
# def really_everything (year):
#     #getting sports reference stuff
#     newdf2=everything ('https://www.sports-reference.com/cbb/postseason/'+f'{year}'+'-ncaa.html')
#     the_year = []
#     #print('before the for loop')
#     for thing in newdf2['first_team']:
#         the_year.append(year)
#     newdf2['year']=the_year
#     #print('right before new loop')
    
    
#     other_team_seed = []
#     i = 0
#     for thing in list(newdf2['team_rank']):
#         if thing == list(newdf2['second_team'])[i]:
#             other_team_seed.append(list(newdf2['first_team'])[i])
#         elif thing == list(newdf2['first_team'])[i]:
#             other_team_seed.append(list(newdf2['second_team'])[i])
#         i+=1
#     #print('made it here')
#     newdf2['other_team_seed']=other_team_seed
#     #print('finished sports reference')
    
#     #getting ESPN stuff
#     df2=merge_espn (year)
#     #print('finished espn')
    
#     #merging
#     possibly2 = pd.merge(newdf2,df2, left_on = 'name', right_on = 'Name')
#     possibly2 = possibly2.sort_values(by ='index' )
#     the_values=find_missing_values (list(possibly2['index']))
#     for thing in the_values:
#         #print(thing)
#         the_name=list(newdf2[(newdf2['index']==thing)]['name'])[0]
#         ESPN_name=ESPN_sports_reference_dict[f"{the_name}"]
#         possibly2=add_to_df (ESPN_name, thing,possibly2, df2, newdf2)
    
#     possibly2.drop(['first_team','second_team'], axis = 1, inplace = True)
#     possibly2.rename(columns={'team_rank':'team_seed'}, inplace = True)
#     possibly2 = possibly2.sort_values(by ='index' )

        
#     return possibly2

In [259]:
def really_everything (year):
    #getting sports reference stuff
    newdf2=everything ('https://www.sports-reference.com/cbb/postseason/'+f'{year}'+'-ncaa.html')
    the_year = []
    #print('before the for loop')
    for thing in newdf2['first_team']:
        the_year.append(year)
    newdf2['year']=the_year
    #print('right before new loop')
    
    

    
    #getting ESPN stuff
    df2=merge_espn (year)
    #print('finished espn')
    
    #merging
    possibly2 = pd.merge(newdf2,df2, left_on = 'name', right_on = 'Name')
    possibly2 = possibly2.sort_values(by ='index' )
    the_values=find_missing_values (list(possibly2['index']))
    for thing in the_values:
        #print(thing)
        the_name=list(newdf2[(newdf2['index']==thing)]['name'])[0]
        ESPN_name=ESPN_sports_reference_dict[f"{the_name}"]
        possibly2=add_to_df (ESPN_name, thing,possibly2, df2, newdf2)
    
    
    other_team_seed = []
    i = 0
    for thing in list(possibly2['team_rank']):
        if thing == list(possibly2['second_team'])[i]:
            other_team_seed.append(list(possibly2['first_team'])[i])
        elif thing == list(possibly2['first_team'])[i]:
            other_team_seed.append(list(possibly2['second_team'])[i])
        i+=1
    #print('made it here')
    possibly2['other_team_seed']=other_team_seed
    #print('finished sports reference')
    
    
    possibly2.drop(['first_team','second_team'], axis = 1, inplace = True)
    possibly2.rename(columns={'team_rank':'team_seed'}, inplace = True)
    possibly2 = possibly2.sort_values(by ='index' )

        
    return possibly2

In [267]:
all_data_not_advanced=really_everything (1985)

https://www.sports-reference.com/cbb/postseason/1985-ncaa.html


In [260]:
#saved_data = all_data_not_advanced

In [270]:
for year in years:
    data=really_everything(year)
    all_data_not_advanced=all_data_not_advanced.append(data, ignore_index=True)

https://www.sports-reference.com/cbb/postseason/1986-ncaa.html
https://www.sports-reference.com/cbb/postseason/1987-ncaa.html
https://www.sports-reference.com/cbb/postseason/1988-ncaa.html
https://www.sports-reference.com/cbb/postseason/1989-ncaa.html
https://www.sports-reference.com/cbb/postseason/1990-ncaa.html
https://www.sports-reference.com/cbb/postseason/1991-ncaa.html
https://www.sports-reference.com/cbb/postseason/1992-ncaa.html
https://www.sports-reference.com/cbb/postseason/1993-ncaa.html
https://www.sports-reference.com/cbb/postseason/1994-ncaa.html
https://www.sports-reference.com/cbb/postseason/1995-ncaa.html
https://www.sports-reference.com/cbb/postseason/1996-ncaa.html
https://www.sports-reference.com/cbb/postseason/1997-ncaa.html
https://www.sports-reference.com/cbb/postseason/1998-ncaa.html
https://www.sports-reference.com/cbb/postseason/1999-ncaa.html
https://www.sports-reference.com/cbb/postseason/2000-ncaa.html
https://www.sports-reference.com/cbb/postseason/2001-nc

In [271]:
all_data_not_advanced.to_csv('all_data_not_advanced.csv')

In [209]:
possibly2=possibly2.append(possibly3, ignore_index=True)

In [265]:
years = []
beg = 1986
for num in range(1,35):
    years.append(beg)
    beg+=1

In [269]:
years

[1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [203]:
stats_not_advanced = {}
stats_not_advanced = pd.DataFrame(stats_not_advanced)

In [204]:
stats_not_advanced

""


In [205]:
years[0]

1985

In [ ]:
stats_not_advanced = {}
for year in years:
    possibly2=really_everything (year)

In [179]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
#possibly2 = possibly2.sort_values(by ='index' )
#possibly2.drop(['first_team','second_team'], axis = 1, inplace = True)
#possibly2.rename(columns={'team_rank':'team_seed'}, inplace = True)
possibly2

,winner,point_difference,result,name,team_seed,win,higher_or_lower_seed,index,year_x,other_team_seed,Name,BLKPG,STPG,APG,AST_TO,_3pm,_3pa,_2pm,_2pa,_2p_perc,PTS,FG_perc,_3p_perc,FT_perc,year_y,ORPG,DRPG,RPG
0,1,15,1,Duke,1,1,1,0,2019,16.0,Duke,6.82,9.47,15.9,1.21,7.3,24.0,790.0,1355.0,0.583,83.5,0.477,0.302,0.690,2019,13.4,28.4,41.8
1,1,15,1,North Dakota State,16,0,0,1,2019,1.0,North Dakota State,2.48,4.82,11.6,1.10,9.5,26.0,513.0,955.0,0.537,73.2,0.456,0.366,0.774,2019,6.6,24.9,31.5
2,9,1,2,VCU,8,0,1,2,2019,9.0,VCU,4.53,7.97,13.9,1.00,7.2,23.0,572.0,1066.0,0.537,71.4,0.442,0.307,0.698,2019,10.8,25.9,36.8
3,9,1,2,UCF,9,1,0,3,2019,8.0,UCF,4.48,5.74,13.3,1.12,6.8,19.0,554.0,1056.0,0.525,72.1,0.463,0.354,0.645,2019,9.8,26.9,36.7
4,12,7,2,Mississippi State,5,0,1,4,2019,12.0,Mississippi State,4.97,8.09,14.4,1.08,8.6,23.0,627.0,1178.0,0.532,77.3,0.472,0.378,0.715,2019,11.7,24.1,35.8
5,12,7,2,Liberty,12,1,0,5,2019,5.0,Liberty,2.68,6.38,14.9,1.35,8.8,24.0,621.0,1061.0,0.585,74.0,0.490,0.367,0.771,2019,7.8,25.0,32.8
6,4,9,1,Virginia Tech,4,1,1,6,2019,13.0,Virginia Tech,2.31,6.66,15.3,1.35,9.6,24.0,517.0,954.0,0.542,74.0,0.475,0.394,0.758,2019,8.9,23.7,32.6
7,4,9,1,Saint Louis,13,0,0,7,2019,4.0,Saint Louis,3.94,7.12,13.0,1.00,5.8,19.0,617.0,1311.0,0.471,67.4,0.419,0.313,0.600,2019,13.8,26.0,39.8
8,6,5,1,Maryland,6,1,1,8,2019,11.0,Maryland,4.75,4.34,13.2,1.01,7.3,21.0,575.0,1120.0,0.513,71.3,0.454,0.353,0.748,2019,11.0,28.2,39.2
9,6,5,1,Belmont,11,0,0,9,2019,6.0,Belmont,3.77,6.84,19.9,1.71,10.5,28.0,662.0,1105.0,0.599,87.4,0.499,0.373,0.738,2019,8.9,29.9,38.9


In [187]:
perhaps=pd.merge(possibly2, adv_stat, how = 'inner', on='name')

In [189]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
perhaps = perhaps.sort_values(by ='index' )
perhaps

,winner,point_difference,result,name,team_seed,win,higher_or_lower_seed,index,year_x,other_team_seed,Name,BLKPG,STPG,APG,AST_TO,_3pm,_3pa,_2pm,_2pa,_2p_perc,PTS,FG_perc,_3p_perc,FT_perc,year_y,ORPG,DRPG,RPG,SRS_adv,SOS_adv,pace_adv,ORtg_adv,eFG_perc_adv
0,1,15,1,Duke,1,1,1,0,2019,16.0,Duke,6.82,9.47,15.9,1.21,7.3,24.0,790.0,1355.0,0.583,83.5,0.477,0.302,0.690,2019,13.4,28.4,41.8,26.90,11.98,73.2,112.6,0.536
4,1,15,1,North Dakota State,16,0,0,1,2019,1.0,North Dakota State,2.48,4.82,11.6,1.10,9.5,26.0,513.0,955.0,0.537,73.2,0.456,0.366,0.774,2019,6.6,24.9,31.5,-4.01,-2.07,66.8,108.5,0.540
5,12,7,2,Mississippi State,5,0,1,4,2019,12.0,Mississippi State,4.97,8.09,14.4,1.08,8.6,23.0,627.0,1178.0,0.532,77.3,0.472,0.378,0.715,2019,11.7,24.1,35.8,15.96,9.07,68.6,111.9,0.546
6,12,7,2,Liberty,12,1,0,5,2019,5.0,Liberty,2.68,6.38,14.9,1.35,8.8,24.0,621.0,1061.0,0.585,74.0,0.490,0.367,0.771,2019,7.8,25.0,32.8,5.27,-3.88,65.4,112.8,0.569
8,4,9,1,Virginia Tech,4,1,1,6,2019,13.0,Virginia Tech,2.31,6.66,15.3,1.35,9.6,24.0,517.0,954.0,0.542,74.0,0.475,0.394,0.758,2019,8.9,23.7,32.6,19.28,7.79,64.4,113.3,0.556
11,4,9,1,Saint Louis,13,0,0,7,2019,4.0,Saint Louis,3.94,7.12,13.0,1.00,5.8,19.0,617.0,1311.0,0.471,67.4,0.419,0.313,0.600,2019,13.8,26.0,39.8,5.06,2.23,67.0,99.4,0.467
12,6,5,1,Maryland,6,1,1,8,2019,11.0,Maryland,4.75,4.34,13.2,1.01,7.3,21.0,575.0,1120.0,0.513,71.3,0.454,0.353,0.748,2019,11.0,28.2,39.2,16.01,10.09,66.6,107.3,0.514
14,6,5,1,Belmont,11,0,0,9,2019,6.0,Belmont,3.77,6.84,19.9,1.71,10.5,28.0,662.0,1105.0,0.599,87.4,0.499,0.373,0.738,2019,8.9,29.9,38.9,9.12,-2.60,74.2,116.2,0.580
15,3,11,1,Yale,14,0,0,11,2019,3.0,Yale,4.32,6.00,17.3,1.29,7.8,21.0,617.0,1101.0,0.560,80.6,0.494,0.370,0.725,2019,8.8,30.2,38.9,5.52,-1.24,72.9,109.7,0.556
16,10,3,2,Louisville,7,0,1,12,2019,10.0,Louisville,3.00,4.45,13.4,1.09,8.6,25.0,543.0,1074.0,0.506,74.5,0.434,0.342,0.775,2019,10.3,28.0,38.2,17.28,11.04,68.4,107.4,0.509


# Merging espn and sports reference

In [ ]:
possibly2 = pd.merge(newdf2,df2, left_on = 'name', right_on = 'Name')

In [ ]:
possibly2 = possibly2.sort_values(by ='index' )

In [ ]:
possibly2 = possibly2.sort_values(by ='index' )
possibly2

In [198]:
#need to run this
def find_missing_values (blah):
    the_values = []
    for thing in list(range(0,126)):
        if thing in blah:
            continue
        else:
            #print(thing)
            the_values.append(thing)
    return the_values

In [ ]:
the_values=find_missing_values (list(possibly2['index']))

In [ ]:
newdf2[(newdf2['index']==31)]


In [ ]:
pd.set_option('display.max_rows', 400)
df2.Name

In [ ]:
df2

In [ ]:
df2[(df2['Name']=='San José St')]

In [67]:
possibly2.head(1)

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year_x,Name,BLKPG,STPG,APG,AST_TO,_3pm,_3pa,_2pm,_2pa,_2p_perc,PTS,FG_perc,_3p_perc,FT_perc,year_y,ORPG,DRPG,RPG
0,1,16,1,15,1,Duke,1,1,1,0,2019,Duke,6.82,9.47,15.9,1.21,7.3,24.0,790.0,1355.0,0.583,83.5,0.477,0.302,0.69,2019,13.4,28.4,41.8


In [69]:
possibly2[(possibly2['name']=='UNC')]

,first_team,second_team,winner,point_difference,result,name,team_rank,win,higher_or_lower_seed,index,year_x,Name,BLKPG,STPG,APG,AST_TO,_3pm,_3pa,_2pm,_2pa,_2p_perc,PTS,FG_perc,_3p_perc,FT_perc,year_y,ORPG,DRPG,RPG
121,1,16,1,15,1,UNC,1,1,1,60,2019,North Carolina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.1,0.464,0.365,0.742,2019,NaN,NaN,NaN
122,1,9,1,8,1,UNC,1,1,1,76,2019,North Carolina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.1,0.464,0.365,0.742,2019,NaN,NaN,NaN
123,1,5,5,4,2,UNC,1,0,1,84,2019,North Carolina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.1,0.464,0.365,0.742,2019,NaN,NaN,NaN


In [70]:
#still working on this - dont use
def add_to_df (name1, num,df, df2, newdf2):
    maybe=df2[(df2['Name']==f'{name1}')]
    maybe2=newdf2[(newdf2['index']==num)]
    dict = {'first_team':list(maybe2['first_team'])[0],'second_team':list(maybe2['second_team'])[0],
            'winner':list(maybe2['winner'])[0],
            'point_difference':list(maybe2['point_difference'])[0],'result':list(maybe2['result'])[0],
            'name':list(maybe2['name'])[0],
            'team_rank':list(maybe2['team_rank'])[0],'win':list(maybe2['win'])[0],
            'higher_or_lower_seed':list(maybe2['higher_or_lower_seed'])[0],
            'index':list(maybe2['index'])[0],'year_x':list(maybe['year'])[0], 'Name': list(maybe['Name'])[0],
            'PTS':list(maybe['PTS'])[0],'year_y':list(maybe['year'])[0],
            'FG_perc':list(maybe['FG_perc'])[0],'_3p_perc':list(maybe['_3p_perc'])[0],
            'FT_perc':list(maybe['FT_perc'])[0],'year_y':list(maybe['year'])[0],
            'BLKPG':list(maybe['BLKPG'])[0],'STPG':list(maybe['STPG'])[0],
            'APG':list(maybe['APG'])[0],'AST_TO':list(maybe['AST_TO'])[0],
            '_3pm':list(maybe['_3pm'])[0],'_3pa':list(maybe['_3pa'])[0],
            '_2pm':list(maybe['_2pm'])[0],'_2pa':list(maybe['_2pa'])[0],
            '_2p_perc':list(maybe['_2p_perc'])[0],'ORPG':list(maybe['ORPG'])[0],
            'DRPG':list(maybe['DRPG'])[0],'RPG':list(maybe['RPG'])[0],}
    possibly2=df.append(dict, ignore_index=True)
    return possibly2

In [ ]:
pd.set_option('display.max_rows', 400)

possibly2 = possibly2.sort_values(by ='index' )
possibly2

In [ ]:
for thing in the_values:
    print(thing)
    the_name=list(newdf2[(newdf2['index']==thing)]['name'])[0]
    ESPN_name=ESPN_sports_reference_dict[f"{the_name}"]
    possibly2=add_to_df (ESPN_name, thing,possibly2)
    

# Making dictionary to reference sports reference and ESPN

In [24]:
ESPN_sports_reference_dict = {}

In [25]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [26]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [27]:
ESPN_sports_reference_dict["UNC"] = "North Carolina"

In [28]:
ESPN_sports_reference_dict["St. Joseph's"] = "Saint Joseph's"

In [29]:
ESPN_sports_reference_dict["Pitt"] = "Pittsburgh"

In [30]:
ESPN_sports_reference_dict["UC-Irvine"] = "UC Irvine"

In [31]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [32]:
ESPN_sports_reference_dict["College of Charleston"] = "Charleston"

In [33]:
ESPN_sports_reference_dict["Miami (FL)"] = "Miami"

In [34]:
ESPN_sports_reference_dict["Loyola (IL)"] = "Loyola Chicago"

In [35]:
ESPN_sports_reference_dict["Penn"] = "Pennsylvania"

In [36]:
ESPN_sports_reference_dict["Mount St. Mary's"] = "Mt. St. Mary's"

In [37]:
ESPN_sports_reference_dict["ETSU"] = "East Tennessee State"

In [38]:
ESPN_sports_reference_dict["UC-Davis"] = "UC Davis"

In [39]:
ESPN_sports_reference_dict["Hawaii"] = "Hawai'i"

In [40]:
ESPN_sports_reference_dict["Cal State Bakersfield"] = "CSU Bakersfield"

In [41]:
ESPN_sports_reference_dict["Albany (NY)"] = "Albany"

In [42]:
ESPN_sports_reference_dict["St. John's (NY)"] = "St. John's"

In [43]:
ESPN_sports_reference_dict["Alabama-Birmingham"] = "UAB"

In [44]:
ESPN_sports_reference_dict["Detroit"] = "Detroit Mercy"

In [45]:
ESPN_sports_reference_dict["UCSB"] = "UC Santa Barbara"

In [46]:
ESPN_sports_reference_dict["St. Peter's"] = "Saint Peter's"

In [47]:
ESPN_sports_reference_dict["Cal State Northridge"] = "CSU Northridge"

In [48]:
ESPN_sports_reference_dict["Cal State Fullerton"] = "CSU Fullerton"

In [49]:
ESPN_sports_reference_dict["Texas-Arlington"] = "UT Arlington"

In [50]:
ESPN_sports_reference_dict["Texas A&M-Corpus Christi"] = "Texas A&M-CC"

In [51]:
ESPN_sports_reference_dict["Southeastern Louisiana"] = "SE Louisiana"

In [52]:
ESPN_sports_reference_dict["McNeese State"] = "McNeese"

In [53]:
ESPN_sports_reference_dict["Prairie View"] = "Prairie View A&M"

In [54]:
ESPN_sports_reference_dict["Nicholls State"] = "Nicholls"

In [55]:
ESPN_sports_reference_dict["San Jose State"] = "San José St"

In [56]:
ESPN_sports_reference_dict["Louisiana-Monroe"] = "UL Monroe"

In [57]:
ESPN_sports_reference_dict["Saint Francis (PA)"] = "St. Francis (PA)"

In [220]:
ESPN_sports_reference_dict["LIU"] = "Long Island University"

In [235]:
ESPN_sports_reference_dict["Southeast Missouri State"] = "SE Missouri St"

# Putting stuff into MySQl so I can use the LIKE operator and merge dataframes

In [ ]:
import mysql.connector
import stuff
cnx = mysql.connector.connect(
    host = stuff.host,
    user = stuff.user,
    passwd = stuff.password)

In [ ]:
cursor = cnx.cursor()

In [ ]:
#insert_stm = """CREATE DATABASE march_madness;"""
#cursor.execute(insert_stm)
#cnx.commit()

In [ ]:
cnx.database = 'march_madness'


In [ ]:
df1.head()

In [ ]:
create_table = """
CREATE TABLE ESPN
(name VARCHAR(225),
PTS FLOAT,
FG_perc FLOAT,
_3p_perc FLOAT,
FT_perc FLOAT);
"""

In [ ]:
cursor.execute(create_table)


In [ ]:
all_data = []
i=0
for thing in (list(df1['name'])):
    new_tuple = ((list(df1['name']))[i], (list(df1['PTS']))[i], (list(df1['FG_perc']))[i], (list(df1['_3p_perc']))[i],
                 (list(df1['FT_perc']))[i])
    all_data.append(new_tuple)
    i+=1

In [ ]:

insert_stm = """INSERT INTO ESPN (name, PTS, FG_perc, _3p_perc, FT_perc) 
VALUES (%s, %s, %s, %s, %s)"""
cursor.executemany(insert_stm, all_data)
cnx.commit()

In [ ]:
create_table = """
CREATE TABLE sports_reference
(first_team INT,
second_team INT,
winner INT,
point_difference INT,
result INT,
name VARCHAR(225),
team_rank INT,
win INT,
relative_seed INT,
index_made INT);
"""

In [ ]:
cursor.execute(create_table)


In [ ]:
newdf2.head(1)

In [ ]:
all_data = []
i=0
for thing in (list(newdf2['name'])):
    new_tuple = ((list(newdf2['first_team']))[i], (list(newdf2['second_team']))[i], (list(newdf2['winner']))[i],
                 (list(newdf2['point_difference']))[i], (list(newdf2['result']))[i], (list(newdf2['name']))[i], (list(newdf2['team_rank']))[i],
                 (list(newdf2['win']))[i], (list(newdf2['higher_or_lower_seed']))[i], (list(newdf2['index']))[i]
                )
    all_data.append(new_tuple)
    i+=1

In [ ]:
all_data[0]

In [ ]:
insert_stm = """INSERT INTO sports_reference (first_team,
second_team,
winner,
point_difference,
result,
name,
team_rank,
win,
relative_seed,
index_made) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
cursor.executemany(insert_stm, all_data)
cnx.commit()

In [ ]:
list(df1['name'])

In [ ]:
df1_sorted = df1.sort_values(by ='name' )

In [ ]:
df1_sorted[df1_sorted(['name']='ohio')]

# testing stuff

In [ ]:
data = {'Name':['Tom', 'nick', 'krish', 'jack'], 'Age':[20, 21, 19, 18], 'index_made':[0,1,2,3]}

In [ ]:
data1 = pd.DataFrame(data)

In [ ]:
data1

In [ ]:
dat = {'Name':['Tomas', 'nickolas', 'krishas', 'jack'], 'p3':[30,40,50,60], 'ft':[8,7,3,2]}

In [ ]:
data2 = pd.DataFrame(dat)

In [ ]:
data2

In [ ]:
ugh=pd.merge(data1,data2,how='inner',on='Name')

In [ ]:
ugh

In [ ]:
maybe=data2[(data2['Name']=='Tomas')]
maybe

In [ ]:
maybe2=data1[(data1['Name']=='Tom')]
maybe2

In [ ]:
dict = {'Name':list(maybe2['Name'])[0],'Age':list(maybe2['Age'])[0],'index_made':list(maybe2['index_made'])[0],
        'p3':list(maybe['p3'])[0],'ft':list(maybe['ft'])[0]}

In [ ]:
ugh=ugh.append(dict, ignore_index=True)

In [ ]:
ugh

In [ ]:
list(maybe2['Name'])[0]

In [ ]:
def add_to_df (name1, name2):
    maybe=data2[(data2['Name']==f'{name1}')]
    maybe2=data1[(data1['Name']==f'{name2}')]
    dict = {'Name':list(maybe2['Name'])[0],'Age':list(maybe2['Age'])[0],'index_made':list(maybe2['index_made'])[0],
        'p3':list(maybe['p3'])[0],'ft':list(maybe['ft'])[0]}
    ugh=ugh.append(dict, ignore_index=True)
